In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [9]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,Unnamed: 0,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Jamestown,US,75.06,broken clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
1,8,Butaritari,KI,81.63,broken clouds,3.0707,172.7902,Isles Sunset Lodge
2,14,Kieta,PG,77.63,overcast clouds,-6.2167,155.6333,Uruna Bay Retreat
3,19,Kavieng,PG,81.70,broken clouds,-2.5744,150.7967,Nusa Island Retreat
4,20,Jalu,LY,83.46,clear sky,29.0331,21.5482,صاحبة الفخامة


In [34]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F </dd>
</dl>
"""

In [35]:
# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
vacation_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]


In [37]:
# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [38]:
# 4a. Add a marker layer for each city to the map.
locations = vacation_df[["Lat", "Lng"]]
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=vacation_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig
# 4b. Display the figure


Figure(layout=FigureLayout(height='420px'))

In [46]:
vacation_df.loc[vacation_df["City"]== "Ixtapa"]

,Unnamed: 0,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
71,228,Ixtapa,MX,86.23,overcast clouds,20.7,-105.2,hotel la terminal


In [56]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df["City"] == "Acapulco"]
vacation_end = vacation_df.loc[vacation_df["City"] == "Acapulco"]
vacation_stop1 = vacation_df.loc[vacation_df["City"]== "Lazaro Cardenas"]
vacation_stop2 = vacation_df.loc[vacation_df["City"]== "Coahuayana"]
vacation_stop3 = vacation_df.loc[vacation_df["City"]== "Ixtapa"]

In [57]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start["Lat"].to_numpy()[0], vacation_start["Lng"].to_numpy()[0]
end = vacation_end["Lat"].to_numpy()[0], vacation_end["Lng"].to_numpy()[0]
stop1 = vacation_stop1["Lat"].to_numpy()[0], vacation_stop1["Lng"].to_numpy()[0] 
stop2 = vacation_stop2["Lat"].to_numpy()[0], vacation_stop2["Lng"].to_numpy()[0]
stop3 = vacation_stop3["Lat"].to_numpy()[0], vacation_stop3["Lng"].to_numpy()[0]

In [58]:
print(start,end,stop1,stop2,stop3)

(16.8634, -99.8901) (16.8634, -99.8901) (17.9583, -102.2) (18.7333, -103.6833) (20.7, -105.2)


In [59]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
city_itinerary = gmaps.directions_layer(
        start, end, waypoints=[stop1, stop2, stop3], travel_mode='DRIVING')

fig.add_layer(city_itinerary)
fig

Figure(layout=FigureLayout(height='420px'))

In [60]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([
    vacation_start,
    vacation_stop1,
    vacation_stop2,
    vacation_stop3
],ignore_index=True)
itinerary_df


,Unnamed: 0,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,575,Acapulco,MX,84.02,broken clouds,16.8634,-99.8901,HS Hotsson Smart Acapulco
1,662,Lazaro Cardenas,MX,84.24,overcast clouds,17.9583,-102.2000,HOTEL QUINTA ANTIGUA
2,259,Coahuayana,MX,89.53,broken clouds,18.7333,-103.6833,El rranchito michuacan de ocampos
3,228,Ixtapa,MX,86.23,overcast clouds,20.7000,-105.2000,hotel la terminal


In [61]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F </dd>
</dl>
"""


In [62]:
# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [63]:
# 11a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))